## Data Cleaning
### 1. Clean housing data, school grduation rate data, and demographic data 

In [2]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from pathlib import Path

In [3]:
# Clean housing data
# Read data
df = pd.read_csv("raw_data/neighborhood_market_tracker.tsv000" ,sep='\t', usecols=['period_end', 'region',
       'city', 'state', 'state_code', 'median_sale_price', 'property_type'])
# Filter data
df_il = df[(df["state_code"] == "IL") & (df["city"] == "Chicago") & (df["property_type"] == 'All Residential')]

# Get only community area names
df_il['region'] = df_il['region'].str.replace('Chicago, IL - ', '')
df_il.rename(columns={'region': 'name'}, inplace=True)

df_il['name'] = df_il['name'].str.replace('The Loop', 'Loop')
df_il['name'] = df_il['name'].str.title()
df_il['name'] = df_il['name'].str.replace('East Riverdale', 'Riverdale')
df_il['name'] = df_il['name'].str.replace('West Rogers Park', 'Rogers Park')
df_il['name'] = df_il['name'].str.replace('Belmont Gardens', 'Belmont Cragin')

# Extract year and month
df_il['period_end'] = pd.to_datetime(df_il['period_end'], format='%Y-%m-%d')
df_il['year'] = df_il['period_end'].dt.year
df_il['month'] = df_il['period_end'].dt.month

# Keep variables needed
df_il = df_il[['year', 'month', 'name', 'median_sale_price']]

# Convert to yearly data
df_il.sort_values(by=['name','year','month'], ascending=True)
df_yearly = df_il.groupby(['year', 'name'], as_index=False)['median_sale_price'].median()

/var/folders/hm/_pzcf0b141vbjfjkskxh1x9h0000gn/T/ipykernel_78974/116435761.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_il['region'] = df_il['region'].str.replace('Chicago, IL - ', '')
/var/folders/hm/_pzcf0b141vbjfjkskxh1x9h0000gn/T/ipykernel_78974/116435761.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_il.rename(columns={'region': 'name'}, inplace=True)
/var/folders/hm/_pzcf0b141vbjfjkskxh1x9h0000gn/T/ipykernel_78974/116435761.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [ ]:
df_yearly.to_csv('housing_price.csv', index=False)

In [ ]:
# Save the data for 2022
df_2022 = df_yearly[(df_yearly["year"] == 2022)]
df_2022.to_csv('cleaned_data/housing_price_2022.csv', index=False)

In [ ]:
# Clean population data
df_pop = pd.read_csv("raw_data/population.csv")
df_pop['name'] = df_pop['name'].str.title()
df_pop.to_csv('cleaned_data/population_lower.csv', index=False)

In [12]:
# Clean school data
df_score = pd.read_csv('raw_data/nwea.csv')
df_score = df_score[(df_score["Subject"] == "READING")]
df_school = pd.read_csv('raw_data/school_community_area.csv')
df_school.rename(columns={'Community Area': 'name'}, inplace=True)
df_school['name'] = df_school['name'].str.title()

df_merge = df_score.merge(df_school, how='left', on='School ID')
df_merge.dropna(inplace=True)
df_merge

,School ID,# Tested,National School Growth Percentile,Subject,name
0,400009,240.0,56.0,READING,Garfield Ridge
1,400011,256.0,64.0,READING,East Garfield Park
3,400017,225.0,50.0,READING,Albany Park
6,400021,332.0,50.0,READING,Austin
7,400023,322.0,66.0,READING,Avalon Park
...,...,...,...,...,...
507,610544,529.0,83.0,READING,West Lawn
508,610548,272.0,68.0,READING,Near West Side
509,610559,476.0,83.0,READING,Brighton Park
510,610586,281.0,90.0,READING,East Side


In [ ]:
def weighted_average(df, value, weight):
    val = df[value]
    wt = df[weight]
    return (val * wt).sum() / wt.sum()

df_weighted = df_merge.groupby('name').apply(weighted_average, 
                                     'National School Growth Percentile', '# Tested')
df_weighted = df_weighted.reset_index(name='percentile')
df_weighted.to_csv('cleaned_data/growth_percentile.csv', index=False )

/var/folders/hm/_pzcf0b141vbjfjkskxh1x9h0000gn/T/ipykernel_69610/2511381834.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weighted = df_merge.groupby('name').apply(weighted_average,


### 2. Merge all the data and turn it to geojson file 

In [3]:
# Read data
df_house = pd.read_csv("cleaned_data/housing_price_2022.csv", usecols=['name', 'median_sale_price'])
df_percentile = pd.read_csv("cleaned_data/growth_percentile.csv")
df_pop = pd.read_csv("cleaned_data/population_lower.csv", usecols=['name', 'population'])

In [4]:
df_percentile['percentile'].describe()

count    76.000000
mean     65.853410
std      13.283384
min      20.661355
25%      57.632728
50%      67.216178
75%      75.409390
max      99.000000
Name: percentile, dtype: float64

In [5]:
df_house['median_sale_price'].describe()

count       262.000000
mean     342206.013359
std      141714.763283
min       51500.000000
25%      240187.500000
50%      321875.000000
75%      419812.500000
max      801250.000000
Name: median_sale_price, dtype: float64

In [ ]:
# Load the CSV
df = pd.read_csv("raw_data/CommAreas_20241126.csv")

# Convert the WKT geometries to shapely objects
df['geometry'] = df['the_geom'].apply(wkt.loads)

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Select only necessary columns for GeoJSON output
gdf = gdf[['COMMUNITY', 'geometry']]

# Rename the columns to fit GeoJSON conventions
gdf.rename(columns={'COMMUNITY': 'name'}, inplace=True)

# Change the 'name' column to title case
gdf['name'] = gdf['name'].str.title()

# Merge information
gdf = gdf.merge(df_house, on="name", how="left")
gdf = gdf.merge(df_percentile, on="name", how="left")
gdf = gdf.merge(df_pop, on="name", how="left")

# Save as GeoJSON
gdf.to_file("www/community_areas.geojson", driver="GeoJSON")

/Users/yihuai/Documents/yihuai/uchi/autumn2/DV/Housing-School/.venv/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
